<a href="https://colab.research.google.com/github/linusms/Hands-on/blob/main/chapter_7_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 개개의 분류기와 앙상블 방법의 성능차이

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist=fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

# 훈련, 검증, 테스트 세트를 한번에 분리해주는 함수는 없으므로
# train_test_split()을 두 번 사용하면 됨

X_train_val, X_test, y_train_val, y_test=train_test_split(mnist.data, mnist.target,test_size=10000, random_state=42)
X_train, X_val, y_train, y_val=train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape)

(50000, 784) (10000, 784) (10000, 784)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# 아래는 GridSearchCV로 최적 파라미터 찾기, but 시간 너무 오래 걸리므로 실제 실행 X
# 아무 변수도 안 넣고 실행해도 5분 걸리니, 엄청 많이 시간 소요되는 코드
rnd_forest=RandomForestClassifier()
rnd_param=[{'max_depth':list(range(10)), 'max_leaf_nodes':list(range(2,100)),'random_state':[42]}]
ex_tree=ExtraTreeClassifier()

ex_param=[{'max_depth':list(range(10)), 'max_leaf_nodes':list(range(2,100)), 'random_state':[42]}]
svm=SVC()
svm_param=[{'C':list(range(10)), 'kernel':'poly', 'degree':list(range(10)), 'random_state':[42]}]
estimators=[rnd_forest, ex_tree, svm]
params=[rnd_param, ex_param, svm_param]

for estimator, param in zip(estimators, params):
  grid_search=GridSearchCV(estimator, param, cv=3, verbose=1)
  grid_search.fit(X_train, y_train)
  print('%s' % estimator)
  print(grid_search.best_params_)
  print(grid_search.best_score_)



In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

# 실행이 오래 걸려서 LinearSVC의 tol을 100으로 맞춤
# 매우 높게 설정했으므로 정확도는 낮을듯

rnd_forest = RandomForestClassifier(n_estimators=100, random_state=42)
ex_tree = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm = LinearSVC(max_iter=100, tol=100, random_state=42)


In [51]:
from sklearn.metrics import accuracy_score

# 실행시간 39초

rnd_forest.fit(X_train, y_train)
rnd_forest.predict(X_val)
y_pred=rnd_forest.predict(X_val)
accuracy_score(y_val, y_pred)

0.9692

In [52]:
# 실행시간 34초

ex_tree.fit(X_train, y_train)
ex_tree.predict(X_val)
y_pred=ex_tree.predict(X_val)
accuracy_score(y_val, y_pred)

0.9715

In [53]:
# 실행시간 1초

svm.fit(X_train, y_train)
y_pred=svm.predict(X_val)
accuracy_score(y_val, y_pred)

0.859

In [54]:
# 실행시간 1분

estimators=[rnd_forest, ex_tree, svm]

from sklearn.ensemble import VotingClassifier
named_estimators = [
    ("random_forest_clf", rnd_forest),
    ("extra_trees_clf", ex_tree),
    ("svm_clf", svm),
]
voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)
voting_clf.score(X_val, y_val)

0.9693

In [55]:
# 원래대로라면 조금 상승했어야 되는데...이유를 모르겠음
# 아마도 충분히 다양하지 않은 예측기 종류(MLPClassifier를 빼서)
# 와 tol 값 설정 때문에 발생한 오차 때문인듯
# 과적합 된 것 같기도 하다

print(voting_clf.get_params())

voting_clf.set_params(svm_clf=None)

voting_clf.score(X_val, y_val)

{'estimators': [('random_forest_clf', RandomForestClassifier(random_state=42)), ('extra_trees_clf', ExtraTreesClassifier(random_state=42)), ('svm_clf', LinearSVC(max_iter=100, random_state=42, tol=100))], 'flatten_transform': True, 'n_jobs': None, 'verbose': False, 'voting': 'hard', 'weights': None, 'random_forest_clf': RandomForestClassifier(random_state=42), 'extra_trees_clf': ExtraTreesClassifier(random_state=42), 'svm_clf': LinearSVC(max_iter=100, random_state=42, tol=100), 'random_forest_clf__bootstrap': True, 'random_forest_clf__ccp_alpha': 0.0, 'random_forest_clf__class_weight': None, 'random_forest_clf__criterion': 'gini', 'random_forest_clf__max_depth': None, 'random_forest_clf__max_features': 'auto', 'random_forest_clf__max_leaf_nodes': None, 'random_forest_clf__max_samples': None, 'random_forest_clf__min_impurity_decrease': 0.0, 'random_forest_clf__min_samples_leaf': 1, 'random_forest_clf__min_samples_split': 2, 'random_forest_clf__min_weight_fraction_leaf': 0.0, 'random_for

0.9693

In [43]:
voting_clf.estimators

[('random_forest_clf', RandomForestClassifier(random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(random_state=42)),
 ('svm_clf', None)]

In [57]:
# 2. 검증 세트에서 예측 만들고 그 결과로 새로운 훈련 세트 만들고 훈련해보기(블렌더 만들기)
import numpy as np

X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)


for index, estimator in enumerate(estimators):


  # 각 열이 서로 다른 예측기의 답을 예측한 것을 샘플마다 행의 값으로 받고있음
    X_val_predictions[:, index] = estimator.predict(X_val)

X_val_predictions

array([[5., 5., 5.],
       [8., 8., 8.],
       [2., 2., 3.],
       ...,
       [7., 7., 7.],
       [6., 6., 6.],
       [7., 7., 7.]], dtype=float32)

In [62]:
# 3개의 예측기에서 나온 10000개의 예측값들로 이루어진 행렬

X_val_predictions.shape

(10000, 3)

In [58]:
# 1번째 레이어의 예측기들의 예측값을 훈련 데이터로 훈련

rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [59]:
# oob 점수로 성능 확인

rnd_forest_blender.oob_score_

0.9703

In [60]:
# 실제 테스트 세트에서 모델 평가해보기

X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [61]:
# 새로 훈련시킨 블렌더 모델의 성능
# 앞서 나온 앙상블 모델보다 더 별로이다

y_pred = rnd_forest_blender.predict(X_test_predictions)

accuracy_score(y_test, y_pred)

0.9661